In [ ]:
# -*- coding: utf-8 -*-
# Copyright (c) 2017-present, Facebook, Inc.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
#

"""
Created on Sat Sep 19 20:55:56 2015

@author: liangshiyu
"""

from __future__ import print_function
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
#import matplotlib.pyplot as plt
import numpy as np
import time
from scipy import misc
import random

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [ ]:
def tpr95(name, file_path):
    #calculate the falsepositive error when tpr is 95%
    T = 1000
    cifar = np.loadtxt(f'{file_path}/confidence_Our_In.txt', delimiter=',')
    other = np.loadtxt(f'{file_path}/confidence_Our_Out.txt', delimiter=',')
    if name == "cifar10": 
        start = 0.1
        end = 0.12 
    if name == "cifar100": 
        start = 0.01
        end = 0.0104    
    gap = (end- start)/100000
    #f = open("./{}/{}/T_{}.txt".format(nnName, dataName, T), 'w')
    Y1 = other[:, 2]
    X1 = cifar[:, 2]
    total = 0.0
    fpr = 0.0
    for delta in np.arange(start, end, gap):
        tpr = np.sum(np.sum(X1 >= delta)) / float(len(X1))
        error2 = np.sum(np.sum(Y1 > delta)) / float(len(Y1))
        if tpr <= 0.9505 and tpr >= 0.9495:
            fpr += error2
            total += 1
    fprNew = fpr/total
            
    return fprNew

In [ ]:
def auroc(name, file_path):
    # calculate our algorithm
    T = 1000
    cifar = np.loadtxt(f'{file_path}/confidence_Our_In.txt', delimiter=',')
    other = np.loadtxt(f'{file_path}/confidence_Our_Out.txt', delimiter=',')
    if name == "cifar10": 
        start = 0.1
        end = 0.12 
    if name == "cifar100": 
        start = 0.01
        end = 0.0104    
    gap = (end- start)/100000
    #f = open("./{}/{}/T_{}.txt".format(nnName, dataName, T), 'w')
    Y1 = other[:, 2]
    X1 = cifar[:, 2]
    aurocNew = 0.0
    fprTemp = 1.0
    for delta in np.arange(start, end, gap):
        tpr = np.sum(np.sum(X1 >= delta)) / float(len(X1))
        fpr = np.sum(np.sum(Y1 >= delta)) / float(len(Y1))
        aurocNew += (-fpr+fprTemp)*tpr
        fprTemp = fpr
    aurocNew += fpr * tpr
    return aurocNew

In [ ]:
def auprIn(name, file_path):
    cifar = np.loadtxt(f'{file_path}/confidence_Our_In.txt', delimiter=',')
    other = np.loadtxt(f'{file_path}/confidence_Our_Out.txt', delimiter=',')
    if name == "cifar10": 
        start = 0.1
        end = 0.12 
    if name == "cifar100": 
        start = 0.01
        end = 0.0104    
    gap = (end- start)/100000
    #f = open("./{}/{}/T_{}.txt".format(nnName, dataName, T), 'w')
    Y1 = other[:, 2]
    X1 = cifar[:, 2]
    auprNew = 0.0
    recallTemp = 1.0
    for delta in np.arange(start, end, gap):
        tp = np.sum(np.sum(X1 >= delta)) / float(len(X1))
        fp = np.sum(np.sum(Y1 >= delta)) / float(len(Y1))
        if tp + fp == 0: continue
        precision = tp / (tp + fp)
        recall = tp
        #precisionVec.append(precision)
        #recallVec.append(recall)
        auprNew += (recallTemp-recall)*precision
        recallTemp = recall
    auprNew += recall * precision
    return auprNew

In [ ]:
def auprOut(name, file_path):
    cifar = np.loadtxt(f'{file_path}/confidence_Our_In.txt', delimiter=',')
    other = np.loadtxt(f'{file_path}/confidence_Our_Out.txt', delimiter=',')
    if name == "cifar10": 
        start = 0.1
        end = 0.12 
    if name == "cifar100": 
        start = 0.01
        end = 0.0104    
    gap = (end- start)/100000
    #f = open("./{}/{}/T_{}.txt".format(nnName, dataName, T), 'w')
    Y1 = other[:, 2]
    X1 = cifar[:, 2]
    auprNew = 0.0
    recallTemp = 1.0
    for delta in np.arange(end, start, -gap):
        fp = np.sum(np.sum(X1 < delta)) / float(len(X1))
        tp = np.sum(np.sum(Y1 < delta)) / float(len(Y1))
        if tp + fp == 0: break
        precision = tp / (tp + fp)
        recall = tp
        auprNew += (recallTemp-recall)*precision
        recallTemp = recall
    auprNew += recall * precision
    return auprNew

In [ ]:
def detection(name, file_path):
    cifar = np.loadtxt(f'{file_path}/confidence_Our_In.txt', delimiter=',')
    other = np.loadtxt(f'{file_path}/confidence_Our_Out.txt', delimiter=',')
    if name == "cifar10": 
        start = 0.1
        end = 0.12 
    if name == "cifar100": 
        start = 0.01
        end = 0.0104    
    gap = (end- start)/100000
    #f = open("./{}/{}/T_{}.txt".format(nnName, dataName, T), 'w')
    Y1 = other[:, 2]
    X1 = cifar[:, 2]
    errorNew = 1.0
    for delta in np.arange(start, end, gap):
        tpr = np.sum(np.sum(X1 < delta)) / float(len(X1))
        error2 = np.sum(np.sum(Y1 > delta)) / float(len(Y1))
        errorNew = np.minimum(errorNew, (tpr+error2)/2.0)
            
    return errorNew

In [ ]:
def metric(model_name='', id_dataset='', ood_dataset='', file_path=''):    
    fprNew = tpr95(id_dataset, file_path)
    errorNew = detection(id_dataset, file_path)
    aurocNew = auroc(id_dataset, file_path)
    auprinNew = auprIn(id_dataset, file_path)
    auproutNew = auprOut(id_dataset, file_path)
    
    print("{:31}{:>22}".format("Neural network architecture:", model_name))
    print("{:31}{:>22}".format("In-distribution dataset:", id_dataset))
    print("{:31}{:>22}".format("Out-of-distribution dataset:", ood_dataset))
    print("")
    print("{:>34}".format("ODIN"))
    print("{:20}{:13.1f}%".format("FPR at TPR 95%:", fprNew*100))
    print("{:20}{:13.1f}%".format("Detection error:", errorNew*100))
    print("{:20}{:13.1f}%".format("AUROC:", aurocNew*100))
    print("{:20}{:13.1f}%".format("AUPR In:", auprinNew*100))
    print("{:20}{:13.1f}%".format("AUPR Out:", auproutNew*100))